# Preparation

<b>Load Libraries</b>

In [11]:
# src
import os, sys
cwd = os.getcwd()
os.chdir('../../..')
path_to_src = os.getcwd()
if path_to_src not in sys.path:
    sys.path.append(path_to_src)
os.chdir(cwd)
from src.jupyter.support import *

# others
from sklearn.preprocessing import TargetEncoder
from warnings import simplefilter
import cloudpickle

<b>Load Libraries</b>

In [5]:
# dataset
materials = prepare_data_to_train('../../../resource/data/cleaned.csv')

<b>MLflow</b>

In [7]:
# connect to local mlflow
mlflow.set_tracking_uri('http://127.0.0.1:5000/')
##
experiment_name = 'Model engineering'
try:
    mlflow.create_experiment(name=experiment_name, artifact_location='../../../resource/mlflow/runs')
except:
    mlflow.set_experiment(experiment_name=experiment_name)

<b>UDC</b>

In [18]:
# udc: auto detect category
class UDC_Transformers(FSBaseTransformer):
    def fit(self, X: np.ndarray, y: np.ndarray=None):
        X, idxes = self.detect_category(X)
        ##
        self.assigned_transformers = [('ohe', OneHotEncoder(drop='first', sparse_output=False), idxes['cat']), 
                                      ('qtt', QuantileTransformer(output_distribution='normal'), idxes['num'])]
        
        super().fit(X, y)

        return self

# Evaluation

In [19]:
# warnings
simplefilter(action='ignore')
 
# feature selector
path_to_sfs = mlflow.artifacts.download_artifacts(f'runs:/1eb663989aa049b089d19baff26658c7/model/sfs.pkl')
with open(path_to_sfs, 'rb') as f:
    sfs = cloudpickle.load(f)
    idxes_to_choose = list(sfs.k_feature_idx_)
feature_selector = FunctionTransformer(lambda X, idxes: X[:, idxes], kw_args={'idxes': idxes_to_choose})

# transformers
transfomers = UDC_Transformers()

# 